- https://towardsdatascience.com/deep-learning-for-nlp-with-pytorch-and-torchtext-4f92d69052f

# Deep Learning For NLP with PyTorch and Torchtext

This is the companion code for my article in medium. There will be no further explanation here, just pure code. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

from torchtext.data import Field 
from torchtext.data import Dataset, Example
from torchtext.data import BucketIterator
from torchtext.vocab import FastText
from torchtext.vocab import CharNGram

import pandas as pd
import numpy as np

In [2]:
embedding = FastText('simple')

.vector_cache/wiki.simple.vec: 293MB [03:23, 1.44MB/s]                               
 99%|█████████▉| 110378/111051 [00:11<00:00, 9684.19it/s]

In [19]:
embedding

In [3]:
# embedding_charngram = CharNGram()

In [4]:
df = pd.DataFrame([
    ['my name is Jack', 'Y'],
    ['Hi I am Jack', 'Y'],
    ['Hello There!', 'Y'],
    ['Hi I am cooking', 'N'],
    ['Hello are you there?', 'N'],
    ['There is a bird there', 'N'],
], columns=['text', 'label'])
df

,text,label
0,my name is Jack,Y
1,Hi I am Jack,Y
2,Hello There!,Y
3,Hi I am cooking,N
4,Hello are you there?,N
5,There is a bird there,N


In [5]:
text_field = Field(
    sequential=True,
    tokenize='basic_english', 
    fix_length=5,
    lower=True
)

label_field = Field(sequential=False, use_vocab=False)

# sadly have to apply preprocess manually
preprocessed_text = df['text'].apply(
    lambda x: text_field.preprocess(x)
)

# load fastext simple embedding with 300d
text_field.build_vocab(
    preprocessed_text, 
    vectors='fasttext.simple.300d'
)

# get the vocab instance
vocab = text_field.vocab

In [6]:
# known token, in my case print 12
print(vocab['are'])
# unknown token, will print 0
print(vocab['crazy'])

12
0


In [7]:
for i, r in df.iterrows():    
    print(list(r.values))

['my name is Jack', 'Y']
['Hi I am Jack', 'Y']
['Hello There!', 'Y']
['Hi I am cooking', 'N']
['Hello are you there?', 'N']
['There is a bird there', 'N']


In [8]:
# we still have to manually handle conversion from categorical to int
ltoi = {l: i for i, l in enumerate(df['label'].unique())}
df['label'] = df['label'].apply(lambda y: ltoi[y])

class DataFrameDataset(Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [9]:
df

,text,label
0,my name is Jack,0
1,Hi I am Jack,0
2,Hello There!,0
3,Hi I am cooking,1
4,Hello are you there?,1
5,There is a bird there,1


In [10]:
train_dataset, test_dataset = DataFrameDataset(
    df=df, 
    fields=(
        ('text', text_field),
        ('label', label_field)
    )
).split()

In [11]:
train_iter, test_iter = BucketIterator.splits(
    datasets=(train_dataset, test_dataset), 
    batch_sizes=(2, 2),
    sort=False
)

In [12]:
class ModelParam(object):
    def __init__(self, param_dict: dict = dict()):
        self.input_size = param_dict.get('input_size', 0)
        self.vocab_size = param_dict.get('vocab_size')
        self.embedding_dim = param_dict.get('embedding_dim', 300)
        self.target_dim = param_dict.get('target_dim', 2)
        
class MyModel(nn.Module):
    def __init__(self, model_param: ModelParam):
        super().__init__()
        self.embedding = nn.Embedding(
            model_param.vocab_size, 
            model_param.embedding_dim
        )
        self.lin = nn.Linear(
            model_param.input_size * model_param.embedding_dim, 
            model_param.target_dim
        )
        
    def forward(self, x):
        features = self.embedding(x).view(x.size()[0], -1)
        features = F.relu(features)
        features = self.lin(features)
        return features

In [13]:
class MyModelWithPretrainedEmbedding(nn.Module):
    def __init__(self, model_param: ModelParam, embedding):
        super().__init__()
        self.embedding = embedding
        self.lin = nn.Linear(
            model_param.input_size * model_param.embedding_dim, 
            model_param.target_dim
        )
        
    def forward(self, x):
        features = self.embedding[x].reshape(x.size()[0], -1)
        features = F.relu(features)
        features = self.lin(features)
        return features

In [14]:
model_param = ModelParam(
    param_dict=dict(
        vocab_size=len(text_field.vocab),
        input_size=5
    )
)
model = MyModel(model_param)
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.01)
epochs = 5


for epoch in range(epochs):
    epoch_losses = list()
    for batch in train_iter:
        optimizer.zero_grad()

        prediction = model(batch.text.T)
        loss = loss_function(prediction, batch.label)

        loss.backward()
        optimizer.step()
        
        epoch_losses.append(loss.item())
    print('train loss on epoch {} : {:.3f}'.format(epoch, np.mean(epoch_losses)))
    
    test_losses = list()
    for batch in test_iter:
        with torch.no_grad():
            optimizer.zero_grad()
            prediction = model(batch.text.T)
            loss = loss_function(prediction, batch.label)
            
            test_losses.append(loss.item())
    
    print('test loss on epoch {}: {:.3f}'.format(epoch, np.mean(test_losses)))

train loss on epoch 0 : 5.631
test loss on epoch 0: 2.989
train loss on epoch 1 : 1.668
test loss on epoch 1: 0.352
train loss on epoch 2 : 0.656
test loss on epoch 2: 0.262
train loss on epoch 3 : 0.522
test loss on epoch 3: 0.194
train loss on epoch 4 : 0.016
test loss on epoch 4: 0.120
